In [15]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> Updating lists func:

    get_participants(event_link)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


# Main( )

In [16]:
data_loc = '../data/'

In [17]:
fighters = pd.read_csv(data_loc + 'update_list_fighters.csv')
fighters.tail(3)

,fighter,fighter_url
14169,Ray Wizard,http://ufcstats.com/fighter-details/ea0ad15545...
14170,Scott Morris,http://ufcstats.com/fighter-details/be9d259be0...
14171,Sean Daugherty,http://ufcstats.com/fighter-details/a683f9ddb7...


#### Extracting data to data frame:

In [ ]:
data = []
column_names = ['name','wins','losses','draws',
                'height','weight','reach','stance','dob',
                'slpm','str_acc','sapm','str_def','td_avg','td_acc','td_def','sub_avg',
                'last_update','last_match','url']

for url in tqdm(fighters.fighter_url):
    data.append(get_fighter_data(url=url))

df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

 43%|████▎     | 6116/14172 [56:04<1:12:03,  1.86it/s]

In [ ]:
data.to_csv(data_loc + 'ufc_fighters_stats.csv', index=0) #test

#### Merge data with existing file: 

In [14]:
org = pd.read_csv(data_loc + 'ufc_fighters_stats.csv')
org.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
1343,Josh Parisian,15,6,0,"6'4""",265lbs.,"79""",Orthodox,"Jun28,1989",4.45,49%,4.50,41%,1.05,33%,48%,0.2,04-17-2023,02-18-2023,http://ufcstats.com/fighter-details/b5da8f7d05...
1344,Jordan Leavitt,11,2,0,"5'9""",155lbs.,"71""",Southpaw,"Jun02,1995",2.64,60%,1.93,56%,2.31,26%,30%,1.2,04-17-2023,02-25-2023,http://ufcstats.com/fighter-details/ac026518a9...
1345,Matt Sayles,8,4,0,"5'7""",155lbs.,"68""",Orthodox,"Feb21,1994",4.83,41%,2.81,61%,0.36,16%,81%,0.7,04-17-2023,12-18-2021,http://ufcstats.com/fighter-details/5b8ba8f551...


In [13]:
data = pd.concat([org, df]).drop_duplicates(subset=['name']).reset_index(drop=True)
data.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
690,Sijara Eubanks,8,7,0,"5'4""",125lbs.,"67""",Orthodox,"Apr27,1985",4.3,43%,3.43,57%,1.9,44%,66%,0.3,04-17-2023,12-18-2021,http://ufcstats.com/fighter-details/4234f0097c...
691,Andre Ewell,17,9,0,"5'8""",145lbs.,"75""",Southpaw,"Jan21,1988",4.28,40%,4.87,51%,0.36,75%,72%,0.1,04-17-2023,12-18-2021,http://ufcstats.com/fighter-details/5f6f049d54...
692,Matt Sayles,8,4,0,"5'7""",155lbs.,"68""",Orthodox,"Feb21,1994",4.83,41%,2.81,61%,0.36,16%,81%,0.7,04-17-2023,12-18-2021,http://ufcstats.com/fighter-details/5b8ba8f551...


#### Saving file:

In [11]:
data.to_csv(data_loc + 'ufc_fighters_stats.csv', index=0)
data.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
3945,Jessica Rakoczy,1,5,0 (1 NC),"5'7""",115lbs.,--,Orthodox,"Apr14,1977",2.5,44%,3.55,64%,0.75,20%,0%,0.8,07-04-2023,25-04-2015,http://ufcstats.com/fighter-details/ffdeb4fbea...
3946,David Onama,10,2,0,"5'11""",145lbs.,"74""",Orthodox,"Jun07,1994",5.3,45%,5.75,51%,1.06,20%,45%,0.3,07-04-2023,13-08-2022,http://ufcstats.com/fighter-details/ffe9703408...
3947,Dillon Smith,0,0,0,"6'0""",170lbs.,--,,--,0.0,0%,0.0,0%,0.0,0%,0%,0.0,07-04-2023,25-09-2009,http://ufcstats.com/fighter-details/ffee353f55...


### >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>